In [1]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf

2022-07-25 20:09:53.198327: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-25 20:09:53.198348: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
DATA_QBITS = 20
LATENT_QBITS = 2
TOTAL_QBITS = DATA_QBITS

In [3]:
dev1 = qml.device('lightning.qubit', wires=TOTAL_QBITS)

2022-07-25 20:09:54.247250: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-25 20:09:54.247273: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-25 20:09:54.247288: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tomskopfbahnhof): /proc/driver/nvidia/version does not exist
2022-07-25 20:09:54.247500: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
@qml.qnode(dev1, diff_method="adjoint")
def qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(TOTAL_QBITS))
    qml.BasicEntanglerLayers(weights, wires=range(TOTAL_QBITS))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(TOTAL_QBITS)]

In [5]:
#fig, ax = qml.draw_mpl(qnode)([1,1,1,1],[[0.1,0.24,0.3,0.14]])
#fig.show()

In [6]:
n_layers = 3
weight_shapes = {"weights": (n_layers, TOTAL_QBITS)}

In [7]:
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=TOTAL_QBITS)

In [8]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

def filter_36(x, y):
    keep = (y == 3) | (y == 6)
    x, y = x[keep], y[keep]
    y = y == 3
    return x,y

x_train, y_train = filter_36(x_train, y_train)
x_test, y_test = filter_36(x_test, y_test)

x_train_threes = np.array(x_train[y_train==True])
x_train = np.array(x_train[y_train==False])

x_test_threes = np.array(x_test[y_test==True])
x_test = np.array(x_test[y_test==False])

In [9]:
x_train = x_train.reshape((x_train.shape[0],28,28,1))
x_test = x_test.reshape((x_test.shape[0],28,28,1))

In [10]:
SCALING = 20

x_train_small = tf.image.resize(x_train, (SCALING,SCALING)).numpy()
x_test_small = tf.image.resize(x_test, (SCALING,SCALING)).numpy()

In [11]:
x_train_small.shape

(5918, 20, 20, 1)

In [12]:
x_train_small = x_train_small.reshape(x_train_small.shape[0],SCALING*SCALING)[:4000]
x_test_small = x_test_small.reshape(x_test_small.shape[0],SCALING*SCALING)

In [13]:
inputs = tf.keras.layers.Input(shape=(400,))
clayer_1 = tf.keras.layers.Dense(32)
dress1 = tf.keras.layers.Dense(DATA_QBITS)
#dress2 = tf.keras.layers.Dense(5)
clayer_2 = tf.keras.layers.Dense(SCALING*SCALING, activation="sigmoid")

In [14]:
model = tf.keras.models.Sequential([inputs, clayer_1, dress1, qlayer, clayer_2])

In [15]:
opt = tf.keras.optimizers.Adam(learning_rate=0.075)
model.compile(opt, loss="binary_crossentropy")

In [16]:
es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2,min_delta=0.0001)
fitting = model.fit(x_train_small, x_train_small, epochs=10, batch_size=32)

Epoch 1/10
  1/125 [..............................] - ETA: 29:10 - loss: 0.6937


KeyboardInterrupt



In [ ]:
model.summary()

In [ ]:
from hep_VQAE import utils as ut

recon = model.predict(x_test_small[:100])

ut.eval_recon(x_test_small[:100].reshape((100,20,20)), recon.reshape((100,20,20)), lognorm=True)